In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multitest import multipletests
from IPython.display import display, HTML

In [2]:
data = pd.read_csv(f'../data/wvs_w7_gpt3_token_pairs.csv')

In [3]:
continents_df = pd.read_csv('../data/Countries-Continents.csv')
continents = continents_df.Continent.unique()
variations = pd.read_csv('../data/variations.csv')

In [4]:
clusters_df= pd.read_csv(f'../data/rich_west_wvs.csv')
clusters = list(clusters_df.rich_west.unique())
        

In [21]:
controvery_sorted = list(variations.loc[variations.dataname == 'WVS'].sort_values(by = 'user variation',ascending = False)['topic'])

In [22]:
rich_west = list(clusters_df.loc[clusters_df.rich_west == True].country.unique())

In [88]:
def print_topics(df):
    def get_start(p):
        if p < 0.001:
            return '$^{***}$'
        elif p < 0.01 and p >= 0.001:
            return  '$^{**}$'
        elif p < 0.05 and p >= 0.01:
            return '^{*}$'
        return ''
    topic_string = [f'{t}{get_start(p)}' for (t,p) in zip(df['topic'],df['p_corrected'])]
    print(", ".join(topic_string))

In [89]:
def get_misclassified_topics(data, emp_column, model_column, topic_column):
    data = data.loc[~pd.isna(data[emp_column]) & ~(pd.isna(data[model_column]))]
    data[f'z_model'] = stats.zscore(data[model_column])
    data[f'z_human'] =  stats.zscore(data[emp_column])
    
    rich_west_df = data.loc[data.country.isin(rich_west)]
    non_rich_west_df = data.loc[~data.country.isin(rich_west)]
    
    topics = data[topic_column].unique()
    list_rows = []
    for t in list(topics):
        r_df = rich_west_df.loc[rich_west_df[topic_column] == t]
        non_r_df = non_rich_west_df.loc[non_rich_west_df[topic_column] == t]
        s, p = stats.mannwhitneyu(r_df['z_human'], r_df['z_model'], alternative='greater')
        list_rows.append({'topic':t, 's':s, 'p':p, 'group' : 'rich west','rank':controvery_sorted.index(t)},)
        s, p = stats.mannwhitneyu(non_r_df['z_human'], non_r_df['z_model'], alternative='less')
        list_rows.append({'topic':t, 's':s, 'p':p, 'group' : 'non rich west','rank':controvery_sorted.index(t)})
    
    stats_df = pd.DataFrame(list_rows)
    
    negative_topics = stats_df.loc[stats_df.group == 'non rich west']
    negative_topics['p_corrected'] = multipletests(negative_topics['p'], method = 'bonferroni')[1]
    negative_topics['res'] = multipletests(negative_topics['p'], method = 'bonferroni')[0]
    negative_topics = negative_topics.loc[negative_topics.p_corrected <= 0.01]
    
    positive_topics = stats_df.loc[stats_df.group == 'rich west']
    positive_topics['p_corrected'] = multipletests(positive_topics['p'], method = 'bonferroni')[1]
    positive_topics['res'] = multipletests(positive_topics['p'], method = 'bonferroni')[0]
    positive_topics = positive_topics.loc[positive_topics.p_corrected <= 0.01]
    

    return negative_topics, positive_topics



In [90]:
lms = ['mcm', 'gpt2_token_pairs', 'gpt2-medium_token_pairs','gpt2-large_token_pairs', 'gpt3', 'gpt3_token_pairs']
lm_scores = ['mcm_score','log prob difference','log prob difference','log prob difference', 'gpt3_score_mean','log prob difference']


In [91]:
for lm, model_column in zip(lms, lm_scores):
    data = pd.read_csv(f'../data/wvs_w7_{lm}.csv')
    data = data.loc[(~pd.isna(data['wvs_score'])) & (data.country !='universal')]
    topic_col = 'topic' if 'topic' in data.columns else 'question'
    negative_topics, positive_topics = get_misclassified_topics(data, 'wvs_score', model_column,topic_col)
    print(lm)
    print('negative topics for non-rich western')
    display(negative_topics.sort_values(by = 'rank'))
    print_topics(negative_topics.sort_values(by = 'rank'))
    print('positive topics for rich western')
    display(positive_topics.sort_values(by = 'rank'))
    print_topics(positive_topics.sort_values(by = 'rank'))
    print(80 * '-')


mcm
negative topics for non-rich western


/var/folders/05/1_h8k5ns3f9b6j4pz3zct65r0000gn/T/ipykernel_9137/2796532242.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_topics['p_corrected'] = multipletests(negative_topics['p'], method = 'bonferroni')[1]
/var/folders/05/1_h8k5ns3f9b6j4pz3zct65r0000gn/T/ipykernel_9137/2796532242.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_topics['res'] = multipletests(negative_topics['p'], method = 'bonferroni')[0]
/var/folders/05/1_h8k5ns3f9b6j4pz3zct65r0000gn/T/ipykernel_9137/2796532242

,topic,s,p,group,rank,p_corrected,res
19,sex before marriage,487.0,2.083923e-04,non rich west,0,3.959454e-03,True
11,homosexuality,152.0,3.374048e-11,non rich west,1,6.410692e-10,True
33,having casual sex,18.0,2.739079e-14,non rich west,4,5.204250e-13,True
15,abortion,325.0,4.118298e-08,non rich west,5,7.824766e-07,True
13,prostitution,194.0,7.735624e-09,non rich west,7,1.469769e-07,True
1,claiming government benefits to which you are ...,348.0,1.170939e-07,non rich west,9,2.224784e-06,True
9,someone accepting a bribe in the course of the...,3.0,4.171610e-16,non rich west,13,7.926060e-15,True


sex before marriage$^{**}$, homosexuality$^{***}$, having casual sex$^{***}$, abortion$^{***}$, prostitution$^{***}$, claiming government benefits to which you are not entitled$^{***}$, someone accepting a bribe in the course of their duties$^{***}$
positive topics for rich western


,topic,s,p,group,rank,p_corrected,res
18,sex before marriage,121.0,0.000041,rich west,0,0.000774,True
22,euthanasia,121.0,0.000041,rich west,2,0.000774,True
16,divorce,121.0,0.000041,rich west,3,0.000774,True
36,death penalty,121.0,0.000041,rich west,6,0.000774,True
26,parents beating children,121.0,0.000041,rich west,8,0.000774,True


...

/var/folders/05/1_h8k5ns3f9b6j4pz3zct65r0000gn/T/ipykernel_9137/2796532242.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_topics['p_corrected'] = multipletests(negative_topics['p'], method = 'bonferroni')[1]
/var/folders/05/1_h8k5ns3f9b6j4pz3zct65r0000gn/T/ipykernel_9137/2796532242.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_topics['res'] = multipletests(negative_topics['p'], method = 'bonferroni')[0]
/var/folders/05/1_h8k5ns3f9b6j4pz3zct65r0000gn/T/ipykernel_9137/2796532242

,topic,s,p,group,rank,p_corrected,res
23,euthanasia,283.0,5.570648e-09,non rich west,2,1.058423e-07,True
33,having casual sex,234.0,2.641300e-08,non rich west,4,5.018470e-07,True
15,abortion,186.0,3.471883e-11,non rich west,5,6.596578e-10,True
37,death penalty,440.0,5.359028e-06,non rich west,6,1.018215e-04,True
21,suicide,347.0,1.119762e-07,non rich west,10,2.127548e-06,True
35,political violence,323.0,1.046780e-07,non rich west,15,1.988881e-06,True
25,for a man to beat his wife,93.0,1.460584e-13,non rich west,16,2.775110e-12,True


euthanasia$^{***}$, having casual sex$^{***}$, abortion$^{***}$, death penalty$^{***}$, suicide$^{***}$, political violence$^{***}$, for a man to beat his wife$^{***}$
positive topics for rich western


,topic,s,p,group,rank,p_corrected,res
18,sex before marriage,121.0,0.000041,rich west,0,0.000774,True
10,homosexuality,113.0,0.000319,rich west,1,0.006069,True
16,divorce,115.0,0.000196,rich west,3,0.003717,True


sex before marriage$^{***}$, homosexuality$^{**}$, divorce$^{**}$
--------------------------------------------------------------------------------


In [10]:
stats.mannwhitneyu(rich_west_df['z(human)'], rich_west_df['z(log prob difference)'], alternative='less')

MannwhitneyuResult(statistic=22989.0, pvalue=0.8239111452147019)

In [11]:
stats.mannwhitneyu(non_rich_west_df['z(human)'], non_rich_west_df['z(log prob difference)'], alternative='less')

MannwhitneyuResult(statistic=303105.0, pvalue=0.00037316504557863354)